In [32]:
import mlflow
mlflow.login()

2024/06/16 17:41:10 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://adb-3088650010345545.5.azuredatabricks.net.


In [33]:
experiment_name = "/Shared/Experiments/Price Prediction California Housing"
try:
    mlflow.create_experiment(
        name = experiment_name,
        tags={
            "mlflow.note.content": "California Housing Price Prediction",
            "project_name": "California Housing Price Prediction",
        }
    )
    experiment = mlflow.get_experiment_by_name(experiment_name)
except mlflow.exceptions.MlflowException:
    print("Experiment already exists")
    experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

Experiment already exists


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/647309157427879', creation_time=1718576734880, experiment_id='647309157427879', last_update_time=1718577566548, lifecycle_stage='active', name='/Shared/Experiments/Price Prediction California Housing', tags={'mlflow.experiment.sourceName': '/Shared/Experiments/Price Prediction '
                                 'California Housing',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.note.content': 'California Housing Price Prediction',
 'mlflow.ownerEmail': 'manuel.gil-m@outlook.com',
 'mlflow.ownerId': '8443236376042681',
 'project_name': 'California Housing Price Prediction'}>

In [34]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [35]:
# load data
data = fetch_california_housing(as_frame=True)

In [36]:
targets = data.target

In [37]:
x_train, x_test, y_train, y_test = train_test_split(data.data, targets, test_size=0.2, random_state=42)

# target
target =data.target_names[0]

# train dataset
train_dataset = x_train.copy()
train_dataset[target] = y_train

# test dataset
test_dataset = x_test.copy()
test_dataset[target] = y_test

In [38]:
# create the pandas dataset for training
pd_train_dataset = mlflow.data.from_pandas(df = train_dataset, predictions=None, targets=target)

# create the pandas dataset for testing
pd_test_dataset = mlflow.data.from_pandas(df = test_dataset, predictions=None, targets=target)

In [44]:
rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)

# get predictions
predictions = rfr.predict(x_test)

with mlflow.start_run(run_name="regressor model") as run:
    print("Run: ", run.info.run_id)
    # log the model
    mlflow.sklearn.log_model(rfr, "model", input_example=x_train.head(5))

    # log the training dataset
    mlflow.log_input(dataset=pd_train_dataset, context="Training")

    # log the testing dataset
    mlflow.log_input(dataset=pd_test_dataset, context="Testing")

    # evaluate the model
    model_uri = f"runs:/{run.info.run_id}/model"
    result = mlflow.evaluate(model = model_uri, data=pd_test_dataset, predictions=None, model_type="regressor")


Run:  3bb8cb130b1d42cb9acade0d08295cfe


c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

2024/06/16 17:58:53 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/06/16 17:58:53 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/06/16 17:58:53 INFO mlflow.models.evaluation.default_evaluator: Shap explainer TreeExplainer is used.
100%|===================| 1997/2000 [01:53<00:00]        2024/06/16 18:00:46 WARNING mlflow.models.evaluation.default_evaluator: Shap evaluation failed. Reason: ExplainerError('Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed